In [ ]:
######### Package Imports #########################################################################
import os, warnings, copy, uuid
# remove warnings from ax
os.environ["PYTHONWARNINGS"] = "ignore"
warnings.filterwarnings('ignore')
import torch, ax
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from ax.modelbridge.transforms.standardize_y import StandardizeY
from ax.modelbridge.transforms.unit_x import UnitX
from ax.modelbridge.transforms.remove_fixed import RemoveFixed
from ax.modelbridge.transforms.log import Log
from ax.core.base_trial import TrialStatus as T
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.plot.slice import plot_slice

from optimpv import *
from optimpv.axBOtorch.axBOtorchOptimizer import axBOtorchOptimizer
from optimpv.DDfits.CVAgent import CVAgent

import pySIMsalabim as sim
from pySIMsalabim.experiments.JV_steady_state import *
from pySIMsalabim.experiments.CV import *

init_notebook_plotting()
warnings.filterwarnings('ignore') 
##############################################################################################
# Define the parameters to be fitted
params = []

mun = FitParam(name = 'l2.mu_n', value = 3e-5, bounds = [1e-5,1e-3], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_n$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mun)

mup = FitParam(name = 'l2.mu_p', value = 8e-4, bounds = [1e-5,1e-3], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_p$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mup)

bulk_tr = FitParam(name = 'l2.N_t_bulk', value = 4e20, bounds = [1e19,1e21], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{T}$', unit='s', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(bulk_tr)

offset_l2_l1 = FitParam(name = 'offset_l2_l1.E_c', value = -0.1, bounds = [-0.2,0.0], values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta E_{L2-L1}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)
params.append(offset_l2_l1)

offset_l2_l3 = FitParam(name = 'offset_l2_l3.E_v', value = 0.1, bounds = [0,0.3], values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta E_{L2-L3}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)
params.append(offset_l2_l3)

Egap_l1 = FitParam(name = 'Egap_l1.E_v', value = 1.6, bounds = [1.55,1.65], type='fixed', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$E_{gap,L1}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)
params.append(Egap_l1)

offset_W_L = FitParam(name = 'offset_W_L.E_c', value = -0.1, bounds = [-0.2,0], type='fixed', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta W_L$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)
params.append(offset_W_L)

eps_r = FitParam(name = 'l2.eps_r', value = 24, bounds = [15,30], type='range', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\epsilon_r$', unit='', axis_type = 'linear', std = 0,encoding = None,force_log = False)
params.append(eps_r)

# R_series = FitParam(name = 'R_series', value = 3e-4, bounds = [1e-4,1e-3], type='fixed', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$R_{series}$', unit='$\Omega$ m$^2$', axis_type = 'log', std = 0,encoding = None,force_log = False)
# params.append(R_series)
#original values
params_orig = copy.deepcopy(params)


In [ ]:
# Set the session paths
session_path_zimt = os.path.join('/home/lecorre/Desktop/pySIMsalabim/', 'SIMsalabim','ZimT')
zimt_device_parameters = os.path.join(session_path_zimt, 'simulation_setup.txt')

# Set the JV parameters

Vmin = 0.2
Vmax = 1.0
V_step = 0.1
freq = 1e5
G_frac = 0
exp_format = 'MottSchottky' # 'MottSchottky' or 'CV'
UUID = str(uuid.uuid4())

cmd_pars = []
for param in params:
    if param.name != 'l2.C_np_bulk' and param.name != 'offset_l2_l1.E_c' and param.name != 'offset_l2_l3.E_v' and param.name != 'Egap_l1.E_v' and param.name != 'offset_W_L.E_c':
        cmd_pars.append({'par':param.name, 'val':str(param.value)})
    elif param.name == 'offset_l2_l1.E_c':
        cmd_pars.append({'par':'l1.E_c', 'val':str(3.9-param.value)})
        vv = 3.9-param.value
  
    elif param.name == 'l2.C_np_bulk':
        cmd_pars.append({'par':'l2.C_n_bulk', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.C_p_bulk', 'val':str(param.value)})

    elif param.name == 'offset_l2_l3.E_v':
        cmd_pars.append({'par':'l3.E_v', 'val':str(5.53-param.value)})
    
    elif param.name == 'Egap_l1.E_v':
        cmd_pars.append({'par':'l1.E_v', 'val': str(vv+param.value)})
    
    elif param.name == 'offset_W_L.E_c':
        cmd_pars.append({'par':'W_L', 'val':str(vv-param.value)})
# print(cmd_pars)

# Run the JV simulation
ret, mess = run_CV_simu(zimt_device_parameters , session_path_zimt, freq, Vmin, Vmax, V_step, G_frac=G_frac,  run_mode=False, output_file = 'CapVol.dat', UUID=UUID, cmd_pars=cmd_pars)


# save data for fitting
X,y = [],[]

# Load the data
data = pd.read_csv(os.path.join(session_path_zimt, 'CapVol_'+UUID+'.dat'), sep=r'\s+')

# Extract the data
X = np.asarray(data['V'].values)
if exp_format == 'MottSchottky':
    y = np.asarray(1/(data['C'].values)**2)
elif exp_format == 'CV':
    y = np.asarray(data['C'].values)

# y2 = np.asarray(data['G'].values)


plt.figure()

plt.plot(X,y)
# plt.xscale('log')
plt.xlabel('Applied Voltage [V]')
if exp_format == 'MottSchottky':
    plt.ylabel('1/C$^2$ [F/m$^2$]$^{-2}$')
elif exp_format == 'CV':
    plt.ylabel('C [F/m$^2$]')

plt.show()


In [3]:
# Define the agents
metric = 'rmsre'
loss = 'log10'

CV = CVAgent(params, X, y, session_path_zimt, Vmin = Vmin, Vmax=Vmax, freq =freq , G_frac = G_frac, metric = metric, loss = loss, threshold=-0.5,minimize=True,exp_format=exp_format, simulation_setup=zimt_device_parameters)


In [4]:
# Define the model kwargs
model_kwargs_list = [{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY]}]

# Define the optimizer
optimizer = axBOtorchOptimizer(params = params, agents = CV, models = ['SOBOL','BOTORCH_MODULAR'],n_batches = [1,5], batch_size = [10,2], ax_client = None,  max_parallelism = 10, model_kwargs_list = None, model_gen_kwargs_list = None, name = 'ax_opti')

In [ ]:
optimizer.optimize()

In [ ]:
ax_client = optimizer.ax_client
best_parameters = ax_client.get_best_parameters()[0]
print(best_parameters)
CV.params_w(best_parameters,CV.params)
print(CV.get_SIMsalabim_clean_cmd(CV.params,sim_type='zimt'))
CV.package_SIMsalabim_files(CV.params,'zimt')

In [ ]:
# plot the evolution of the optimization
render(ax_client.get_contour_plot(param_x="l2.mu_n", param_y="l2.mu_p", metric_name=optimizer.all_metrics[0]))

model = ax_client.generation_strategy.model

render(plot_slice(model=model, param_name="l2.mu_n", metric_name=optimizer.all_metrics[0]))
render(plot_slice(model=model, param_name="l2.mu_p", metric_name=optimizer.all_metrics[0]))
render(plot_slice(model=model, param_name="l2.eps_r", metric_name=optimizer.all_metrics[0]))
# render(plot_slice(model=model, param_name="l2.C_np_bulk", metric_name=metric))



In [ ]:
data = ax_client.experiment.fetch_data()

plt.plot(np.minimum.accumulate(data.df["mean"]), label="Best value seen so far")

plt.xlabel("Number of iterations")
plt.ylabel('log of ' + optimizer.all_metrics[0])


In [9]:
# create dic with keys same as ax_client.experiment.trials[0].arm.parameters
dumdic = {}
for key in ax_client.experiment.trials[0].arm.parameters.keys():
    dumdic[key] = []

# fill the dic with the values of the parameters
for i in range(len(ax_client.experiment.trials)):
    if ax_client.experiment.trials[i].status == T.COMPLETED:
        for key in ax_client.experiment.trials[i].arm.parameters.keys():
            dumdic[key].append(ax_client.experiment.trials[i].arm.parameters[key])

data = ax_client.experiment.fetch_data().df
# print(data)
mse = data['mean']
dumdic['mse'] = mse

df = pd.DataFrame(dumdic)



In [ ]:
# get name of all parameters that are not 'fixed'
names = []
log_scale = []
axis_limits = []
for p in params:
    if p.type != 'fixed':
        names.append(p.name)
        log_scale.append(p.axis_type == 'log')
        axis_limits.append(p.bounds)


# Get all combinations of names
comb = list(combinations(names, 2))

# Determine the grid size
n = len(names)
fig, axes = plt.subplots(n, n, figsize=(15, 15))

# Plot each combination in the grid
for i, xx in enumerate(names):
    for j, yy in enumerate(names):
        xval = np.nan
        yval = np.nan

        for p in params_orig:
            if p.name == xx:
                xval = p.value
            if p.name == yy:
                yval = p.value

        ax = axes[i, j]
        if i == j:
            # kde plot on the diagonal
            sns.kdeplot(x=yy, data=df, ax=ax, fill=True, thresh=0, levels=100, cmap="rocket", color="#03051A", log_scale=log_scale[names.index(xx)])

            ax.axvline(x=yval, color='r', linestyle='-')
            # put point at the best value top of the axis
           

            if log_scale[names.index(yy)]:
                ax.set_xscale('log')
                ax.set_xlim(axis_limits[names.index(yy)])
            else:
                ax.set_xlim(axis_limits[names.index(yy)])
            
            # put x label on the top
            # except for the last one
            if i < n - 1:
                ax.xaxis.set_label_position('top')
                ax.xaxis.tick_top()

        elif i > j:
            kind = 'kde'
            if kind == 'scatter':
                sns.scatterplot(x=yy, y=xx, data=df, ax=ax, color="#03051A")
                ax.set_xscale('log')
                ax.set_yscale('log')
            else:
                sns.kdeplot(x=yy, y=xx, data=df, ax=ax, fill=True, thresh=0, levels=100, cmap="rocket", color="#03051A", log_scale=(log_scale[names.index(yy)], log_scale[names.index(xx)]))

            xval = np.nan
            yval = np.nan
            for p in params_orig:
                if p.name == xx:
                    xval = p.value
                elif p.name == yy:
                    yval = p.value

            # Plot as line over the full axis
            ax.axhline(y=xval, color='r', linestyle='-')
            ax.axvline(x=yval, color='r', linestyle='-')
            ax.scatter(best_parameters[yy], best_parameters[xx], c='y', marker='*', s=200, zorder=10)
            
            ax.set_xlim(axis_limits[names.index(yy)])
            ax.set_ylim(axis_limits[names.index(xx)])
        else:
            ax.set_visible(False)

        if j > 0:
            ax.set_yticklabels([])
            # remove the y axis label
            ax.set_ylabel('')
        if i < n - 1:
            ax.set_xticklabels([])
            # remove the x axis label
            ax.set_xlabel('')

        if i == n - 1:
            ax.set_xlabel(yy)
        if j == 0:
            ax.set_ylabel(xx)

plt.tight_layout()
plt.show()

In [ ]:
# rerun the simulation with the best parameters
yfit = CV.run(parameters=ax_client.get_best_parameters()[0],X=X,exp_format=exp_format)

plt.figure(figsize=(10,10))
plt.plot(X,y,label='data')
plt.plot(X,yfit,label='fit',linestyle='--')
plt.xscale('log')
plt.xlabel('Frequency (Hz)')
if exp_format == 'MottSchottky':
    plt.ylabel('1/C$^2$ [F/m$^2$]$^{-2}$')
elif exp_format == 'CV':
    plt.ylabel('C [F/m$^2$]')
plt.legend()
plt.show()

In [12]:
# Clean up the output files (comment out if you want to keep the output files)
sim.clean_all_output(session_path_zimt)
sim.clean_up_output('CapVol',session_path_zimt)
sim.delete_folders('tmp',session_path_zimt)